This notebook was executed in Google Colab

# Install Dependencies

In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 129.2 MB/s eta 0:00:00


In [2]:
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.7 MB/s eta 0:00:00


In [3]:
pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 12.8 MB/s eta 0:00:00


In [4]:
pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 16.3 MB/s eta 0:00:00


# Token Dependencies

In [5]:
from huggingface_hub import notebook_login

notebook_login()

# Mount Drive

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd
import json
import numpy as np
from transformers import AutoTokenizer, BertForQuestionAnswering, TrainingArguments, Trainer, DefaultDataCollator
import torch
from evaluate import load
from datasets import Dataset

# Read SQuAD format

In [8]:
ID_COLUMN = 'id'
CONTEXT_COLUMN = 'context'
QUESTION_COLUMN = 'question'
ANSWER_COLUMN = 'answers'
CONTEXT_ID_COLUMN = 'c_id'


# based on: https://www.kaggle.com/code/sanjay11100/squad-stanford-q-a-json-to-pandas-dataframe
def squad_json_to_dataframe(file_path, record_path=['data','paragraphs','qas','answers']):
    """
    input_file_path: path to the squad json file.
    record_path: path to deepest level in json file default value is
    ['data','paragraphs','qas','answers']
    """
    file = json.loads(open(file_path).read())
    js = pd.json_normalize(file, record_path)
    m = pd.json_normalize(file, record_path[:-1])
    r = pd.json_normalize(file,record_path[:-2])
    idx = np.repeat(r[CONTEXT_COLUMN].values, r.qas.str.len())
    m[CONTEXT_COLUMN] = idx
    data = m[[ID_COLUMN,QUESTION_COLUMN,CONTEXT_COLUMN,ANSWER_COLUMN]].set_index(ID_COLUMN).reset_index()
    data[CONTEXT_ID_COLUMN] = data[CONTEXT_COLUMN].factorize()[0]
    return data

In [9]:
kolzchut_train_df = squad_json_to_dataframe('/content/train-qa.json')
kolzchut_valid_df = squad_json_to_dataframe('/content/valid-qa.json')
kolzchut_test_df = squad_json_to_dataframe('/content/test-qa.json')

In [10]:
kolzchut_train_df.head()

,id,question,context,answers,c_id
0,6aaedff8-67f4-4938-9b49-c587d301a9f8,מה מהווה סיום תקופת ההודעה המוקדמת בין מעסיק ל...,יחסי העובד-מעסיק בין הצדדים מסתיימים עם תום תק...,"[{'answer_start': 73, 'text': 'בסיום יחסי העבו...",0
1,78f79e7e-5e6c-4955-8ffc-da961620261a,מהם החובות של העובד במהלך תקופת ההודעה המוקדמת?,יחסי העובד-מעסיק בין הצדדים מסתיימים עם תום תק...,"[{'answer_start': 249, 'text': 'להעמיד את עצמו...",0
2,9b77cc1e-edeb-4fe6-a9be-759a205078a5,מה המעסיק חייב להעניק לעובד?,יחסי העובד-מעסיק בין הצדדים מסתיימים עם תום תק...,"[{'answer_start': 182, 'text': 'כל תנאי העבודה...",0
3,3b82dcfb-3fd3-4cfa-bc63-7848bd275350,מהו משמעות החובה להודיע לעובד מוקדם על פיטורים...,יחסי העובד-מעסיק בין הצדדים מסתיימים עם תום תק...,"[{'answer_start': 348, 'text': 'לתת לעובד הודע...",0
4,6d19dcc1-d5e9-4ace-8e37-427cbdd12bde,מתי מחייבת החובה לתת הודעה מוקדמת לעובד במקרה ...,יחסי העובד-מעסיק בין הצדדים מסתיימים עם תום תק...,"[{'answer_start': 379, 'text': 'בעת פיטורים'}]",0


In [11]:
len(kolzchut_train_df), len(kolzchut_valid_df) , len(kolzchut_test_df) 

(400, 25, 25)

# Evaluating the baseline results

In [12]:
tokenizer = AutoTokenizer.from_pretrained("tdklab/hebert-finetuned-hebrew-squad")
heBert = BertForQuestionAnswering.from_pretrained("tdklab/hebert-finetuned-hebrew-squad")
squad_metric = load("squad")

In [13]:
# predict sentece for each context-question pair with given model

def predict(context, question, model, tokenizer = tokenizer):
    """
    Predict the sentence for each context-question pair using the given model.
    
    Args:
        context (str): Context paragrpah.
        question (str): Question sentence.
        model (BertForQuestionAnswering): The pre-trained model used for prediction. 
        tokenizer (BertTokenizerFast): The tokenizer used for encoding the input.
    Returns:
        str: The predicted sentence.
    """
    inputs = tokenizer(question, context, return_tensors="pt")
    with torch.no_grad():
        # calculate outputs
        outputs = model(**inputs)

    # get answer span indexes
    answer_start_index = outputs.start_logits.argmax()
    answer_end_index = outputs.end_logits.argmax()

    predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
    predicted_sentence = tokenizer.decode(
    [i for i in predict_answer_tokens if i < tokenizer.vocab_size])

    return predicted_sentence

In [14]:
PREDICTION_TEXT_COLUMN = 'prediction_text'
ANSWER_START_COLUMN = 'answer_start'
TEXT_COLUMN = 'text'

def predictionEvaluateFormat(contexts,questions,answers,model):
    """
    Returns predictions in the SQuAD metric evaluation format.

    Args:
        contexts (list): List of context paragraphs strings.
        questions (list): List of question strings.
        answers (list): List of dictionaries representing answers with 'answer_start' and 'text' keys.
        model (Model): The question-answering model used for predictions.

    Returns:
        predictions (list) and refrences (list) in the SQuAD metric evaluation format.
    """
    predictions = [
                  {
                    PREDICTION_TEXT_COLUMN: predict(context,question,model),
                    ID_COLUMN: str(idx),
                  } 
              for idx, (context, question,answer) in enumerate(zip(contexts,questions,answers))
                ]
    references = [
                  {
                  ANSWER_COLUMN: {
                      ANSWER_START_COLUMN : [answer[0][ANSWER_START_COLUMN]],
                      TEXT_COLUMN: [answer[0][TEXT_COLUMN]]
                  },
                  ID_COLUMN: str(idx)
                  } 
              for idx, (context, question,answer) in enumerate(zip(contexts,questions,answers))
              ]
    return predictions,references

### Predict the results without fine-tuning

In [15]:
def computeSquadMetric(dataframe, model):
    """
    Compute the exact match and F1 scores of predictions.

    Args:
        dataframe (DataFrame): Input DataFrame containing contexts, questions and answers.
        model (BertForQuestionAnswering): The model used for predictions.

    Returns:
        dict: Results of the SQuAD metric computation of exact match and F1 scores.
    """
    predictions,references = predictionEvaluateFormat(dataframe[CONTEXT_COLUMN], dataframe[QUESTION_COLUMN], dataframe[ANSWER_COLUMN],chosen_model)
    results = squad_metric.compute(predictions=predictions, references=references)
    return results

In [16]:
dataframe = kolzchut_train_df
chosen_model = heBert

computeSquadMetric(dataframe, chosen_model)

{'exact_match': 43.25, 'f1': 60.714987834978146}

In [17]:
dataframe = kolzchut_valid_df
chosen_model = heBert

computeSquadMetric(dataframe, chosen_model)

{'exact_match': 40.0, 'f1': 65.60701750197656}

In [18]:
dataframe = kolzchut_test_df
chosen_model = heBert

computeSquadMetric(dataframe, chosen_model)

{'exact_match': 48.0, 'f1': 65.97142857142856}

# Encoding the dataset

In [19]:
# convert pandas DataFrame to torch.DataSet for preprocessing
kolzchut_train_ds = Dataset.from_pandas(kolzchut_train_df)
kolzchut_valid_ds = Dataset.from_pandas(kolzchut_valid_df)
kolzchut_test_ds = Dataset.from_pandas(kolzchut_test_df)

# preprocessing the dataset

In [20]:
# taken from https://huggingface.co/docs/transformers/tasks/question_answering
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i][0]
        start_char = answer["answer_start"]
        end_char = answer["answer_start"] + len(answer["text"])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [21]:
# taken from https://huggingface.co/docs/transformers/tasks/question_answering
tokenized_squad_train = kolzchut_train_ds.map(preprocess_function, batched=True)
tokenized_squad_valid = kolzchut_valid_ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

# Fine-Tuning  HeBert Model

In [22]:
def get_model():
    model = BertForQuestionAnswering.from_pretrained("tdklab/hebert-finetuned-hebrew-squad")
    model.train()
    model.to(torch.device('cuda'))
    return model

# define constants for hyperparameters
MODEL_OUTPUT_DIR = "kolzchutQA-trainer"
LEARNING_RATE = 1e-5
PER_DEVICE_TRAIN_BATCH_SIZE = 16
NUM_TRAIN_EPOCHS = 4
PER_DEVICE_EVAL_BATCH_SIZE = 16

training_args = TrainingArguments(
    output_dir= "kolzchutQA-trainer",
    evaluation_strategy= "epoch",
    save_strategy = "no" ,
    learning_rate= LEARNING_RATE,
    per_device_train_batch_size= PER_DEVICE_TRAIN_BATCH_SIZE,
    num_train_epochs= NUM_TRAIN_EPOCHS,
    per_device_eval_batch_size= PER_DEVICE_EVAL_BATCH_SIZE,
    push_to_hub= False,
)

trainer = Trainer(
    model_init=get_model,
    args=training_args,
    train_dataset=tokenized_squad_train,
    eval_dataset=tokenized_squad_valid,
    tokenizer=tokenizer,
    data_collator=DefaultDataCollator(),
)


In [23]:
#define hyperparameter search space and function
def hp_search(trial):
    return { 
        "learning_rate": trial.suggest_float("learning_rate", 4e-6 , 1e-5, step = 2e-6),
        'per_device_train_batch_size' : trial.suggest_int("per_device_train_batch_size", 8 , 16 , step=4)
    }

In [24]:
# perform hyperparameter search
trainer.hyperparameter_search(direction="minimize",backend="optuna",hp_space=hp_search, n_trials=15)

# train the model
trainer.train()

[I 2023-05-29 20:56:43,399] A new study created in memory with name: no-name-41a318ba-5ffa-4ab0-bcb1-113a332bb219
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,2.408751
2,No log,1.901770
3,No log,1.753444
4,No log,1.718774


[I 2023-05-29 20:58:38,092] Trial 0 finished with value: 1.7187738418579102 and parameters: {'learning_rate': 4e-06, 'per_device_train_batch_size': 8}. Best is trial 0 with value: 1.7187738418579102.


Epoch,Training Loss,Validation Loss
1,No log,2.764499
2,No log,2.262573
3,No log,2.032381
4,No log,1.974816


[I 2023-05-29 21:00:27,849] Trial 1 finished with value: 1.974815845489502 and parameters: {'learning_rate': 4e-06, 'per_device_train_batch_size': 16}. Best is trial 0 with value: 1.7187738418579102.


...

[I 2023-05-29 21:20:15,283] Trial 13 finished with value: 1.4697421789169312 and parameters: {'learning_rate': 9.999999999999999e-06, 'per_device_train_batch_size': 8}. Best is trial 3 with value: 1.4697421789169312.


Epoch,Training Loss,Validation Loss
1,No log,1.688539
2,No log,1.501147
3,No log,1.470955
4,No log,1.469742


[I 2023-05-29 21:22:10,839] Trial 14 finished with value: 1.4697421789169312 and parameters: {'learning_rate': 9.999999999999999e-06, 'per_device_train_batch_size': 8}. Best is trial 3 with value: 1.4697421789169312.


Epoch,Training Loss,Validation Loss
1,No log,1.688539
2,No log,1.501147
3,No log,1.470955
4,No log,1.469742


TrainOutput(global_step=200, training_loss=2.051369323730469, metrics={'train_runtime': 114.2062, 'train_samples_per_second': 14.01, 'train_steps_per_second': 1.751, 'total_flos': 313556108083200.0, 'train_loss': 2.051369323730469, 'epoch': 4.0})

In [25]:
# save best model for future use
trainer.save_model("kolZchutQA")

## Evaluation the fine-tuning results

In [26]:
# load saved best model from training
kolZchutQA = BertForQuestionAnswering.from_pretrained("/content/kolZchutQA")

In [27]:
dataframe = kolzchut_valid_df
chosen_model = kolZchutQA

computeSquadMetric(dataframe, chosen_model)

{'exact_match': 44.0, 'f1': 73.21700950378298}

In [28]:
dataframe = kolzchut_test_df
chosen_model = kolZchutQA

computeSquadMetric(dataframe, chosen_model)

{'exact_match': 52.0, 'f1': 75.10547295841414}

## Print example results

In [29]:
def show_kolzchutqa_prediction(index,model = kolZchutQA , dataframe = kolzchut_test_df):
    context = dataframe[CONTEXT_COLUMN][index]
    question = dataframe[QUESTION_COLUMN][index]
    answer = dataframe[ANSWER_COLUMN][index]
    print("Question: ", question)
    print(question)
    print("Question: ", question)
    print(answer)
    print("Predicted answer: ", predict(context, question, model))


In [30]:
show_kolzchutqa_prediction(10)

Question:  מאיזה רשות העובד זכאי לתשלום דמי אימוץ?
מאיזה רשות העובד זכאי לתשלום דמי אימוץ?
Question:  מאיזה רשות העובד זכאי לתשלום דמי אימוץ?
[{'answer_start': 94, 'text': 'מהמוסד לביטוח לאומי'}]
Predicted answer:  


In [31]:
show_kolzchutqa_prediction(11)

Question:  מה היתרונות של קניית מנות נוספות מאותו תורם לטיפול נוסף?
מה היתרונות של קניית מנות נוספות מאותו תורם לטיפול נוסף?
Question:  מה היתרונות של קניית מנות נוספות מאותו תורם לטיפול נוסף?
[{'answer_start': 732, 'text': 'שלא תמיד הטיפול הראשון מצליח ויש צורך במנות נוספות לטיפולים נוספים'}]
Predicted answer:  ניתן לקנות כמה מנות זרע מאותו תורם, וזאת מכיוון שלא תמיד הטיפול הראשון מצליח ויש צורך במנות


In [32]:
show_kolzchutqa_prediction(12)

Question:  מהם המאפיינים הפיזים של התורם שיימסרו לאישה?
מהם המאפיינים הפיזים של התורם שיימסרו לאישה?
Question:  מהם המאפיינים הפיזים של התורם שיימסרו לאישה?
[{'answer_start': 327, 'text': 'גובה, מבנה גוף, גוון עור, צבע עיניים וצבע שיער'}]
Predicted answer:  גובה, מבנה גוף, גוון עור, צבע עיניים וצבע שיער


In [33]:
# save best model to google drive in zip format
!zip -r /content/drive/MyDrive/NLP-project/Final_Model.zip /content/kolZchutQA

updating: content/kolZchutQA/ (stored 0%)
updating: content/kolZchutQA/special_tokens_map.json (deflated 42%)
updating: content/kolZchutQA/config.json (deflated 46%)
updating: content/kolZchutQA/tokenizer_config.json (deflated 45%)
updating: content/kolZchutQA/vocab.txt (deflated 63%)
updating: content/kolZchutQA/training_args.bin (deflated 48%)
updating: content/kolZchutQA/tokenizer.json (deflated 74%)
updating: content/kolZchutQA/pytorch_model.bin (deflated 7%)
